In [81]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import re
from konlpy.tag import Okt
from collections import Counter

**Functions**

In [83]:
#한글 제외 단어 삭제

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [84]:
#명사추출 colname should be content

def extract_nouns(data):
  okt = Okt()
  corpus = "".join(data.content.tolist())
  nouns = okt.nouns(apply_regular_expression(corpus))
  return nouns


In [85]:
#add_word should be string ex: "abc"

def add_stopwords(add_word):
  stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
  ebike_stopwords = ['자전거', '생각', '정도', '아주', '사은', '타고', '모두','정말','타고','조금','바로','보고','일단','매우','느낌','하나','다시','진짜','한번','대비','자체','약간','사람','듭니','전기자전거','생각','정도','아주','사은','정말','타고','설명','조금','바로','마음','보고','일단','매우','느낌','하나','이상','다시','진짜','한번','대비','자체','약간','사람','듭니','구매','제품','생각','아주','문제','구입','바로','사장','바로','마음','처음','사용','그냥','부분','전기']
  ebike_stopwords.append(add_word)
  for word in ebike_stopwords:
    stopwords.append(word)
  return stopwords


In [86]:
def rem_stopword_nouns(nouns, stopwords):
  remove_stopword_nouns =[]
  for words in nouns:
    if words not in stopwords:
      remove_stopword_nouns.append(words)
  return remove_stopword_nouns

In [87]:
def count_word_more_than1(data):
  counter = Counter(data)
  available_counter = Counter({x: counter[x] for x in counter if len(x) > 1})
  return available_counter


In [89]:
#path - 경로 : string / add- 불용어 추가 : string / number :숫자

def word_counting(datafram,add,number):
  df = datafram
  extracted_nouns = extract_nouns(df)
  added_stopwords = add_stopwords(add)
  removed_stopword_nouns = rem_stopword_nouns(extracted_nouns, added_stopwords)
  preprocessed_nouns = count_word_more_than1(removed_stopword_nouns)
  return preprocessed_nouns.most_common(int(number))


**Running codes**

In [91]:
bic_df = pd.read_csv("bic_price.csv",encoding = "utf-8")
bic_df['가격대'].drop_duplicates()

0         저가
837       중가
2610      고가
3879    프리미엄
Name: 가격대, dtype: object

In [92]:
bic_df_low = bic_df[bic_df['가격대']=='저가']
bic_df_middle = bic_df[bic_df['가격대']=='중가']
bic_df_high = bic_df[bic_df['가격대']=='고가']
bic_df_premium = bic_df[bic_df['가격대']=='프리미엄']

In [93]:
wordcount_low = word_counting(bic_df_low, "자전거", 50)
wordcount_middle = word_counting(bic_df_middle,"자전거",50)
wordcount_high = word_counting(bic_df_high,"자전거",50)
wordcount_premium = word_counting(bic_df_premium,"자전거",50)

In [98]:
wordcount = pd.DataFrame(list(zip(wordcount_low , wordcount_middle, wordcount_high,wordcount_premium)),
               columns =['저가', '중가','고가','프리미엄'])
wordcount.to_csv('빈도수 분석',encoding= 'utf-8-sig')

In [99]:
wordcount.to_csv('빈도수 분석.csv',encoding= 'utf-8-sig')

In [100]:
wordcount

,저가,중가,고가,프리미엄
0,"(조립, 839)","(배송, 1206)","(배송, 875)","(배송, 196)"
1,"(배송, 671)","(조립, 767)","(조립, 330)","(조립, 95)"
2,"(배터리, 443)","(배터리, 586)","(배터리, 321)","(디자인, 64)"
3,"(가격, 355)","(안장, 382)","(안장, 209)","(배터리, 59)"
4,"(가성, 320)","(출퇴근, 282)","(출퇴근, 187)","(주행, 49)"
5,"(안장, 300)","(속도, 277)","(포장, 180)","(출퇴근, 47)"
6,"(출퇴근, 278)","(페달, 267)","(브레이크, 167)","(주문, 43)"
7,"(주행, 181)","(브레이크, 261)","(주행, 166)","(직접, 41)"
8,"(디자인, 178)","(디자인, 260)","(주문, 151)","(최고, 40)"
9,"(바퀴, 171)","(바퀴, 250)","(디자인, 143)","(포장, 37)"
